In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Install libraries
```%pip install -q youtube-transcript-api langchain-community langchain-openai faiss-cpu tiktoken python-dotenv```

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## Step 1a - Indexing (Document Ingestion)

In [3]:
video_id = "La385-EDmWw" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])
    # {text: , start: , duration: }, ...

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    # print(transcript)   # full video caption

except TranscriptsDisabled:
    print("No captions available for this video.")

In [4]:
transcript_list[0:3]

[{'text': 'Software engineering is getting crushed',
  'start': 0.08,
  'duration': 3.92},
 {'text': 'by AI and Lead Code the platform that is',
  'start': 1.839,
  'duration': 3.92},
 {'text': 'used for software engineering interviews',
  'start': 4.0,
  'duration': 3.6}]

## Step 1b - Indexing (Text Splitting)

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [6]:
len(chunks)

17

In [7]:
chunks[10]

Document(metadata={}, page_content='and the candidate gets paid. So this is the true win-win scenario that lead code was striving to be. Plus startup interviews they actually recommend that you use AI or any tools that you would have access to on the job because if it helps you write better software once again that\'s a win-win. So if you\'re trying to prepare for this new style of interview, forget grinding leap code and focus on building real things. Start by using AI tools like GitHub C-Pilot, Cursor or Warp to accelerate your coding workflow and work smarter. Use platforms like Replet or Versel to quickly develop full stack applications. But whatever you do, make sure to build projects that solve actual problems, not just another coding project for the sake of doing a coding project. I am tired of to-do list apps. Oh, but what if I have no idea what coding project I can do? Try this. Go to a local mom and pop store and ask them, "Is there any annoying, time-consuming things that yo

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embeddings)

In [9]:
vector_store.index_to_docstore_id

{0: '688efe80-1239-4b77-bf07-b37e2c2062e2',
 1: '2acabf9d-1277-45f3-b379-f1c338467739',
 2: '638a0580-2292-4e78-aeb3-9b21f9051a47',
 3: '672b9c0a-cc3c-40fe-b148-bc4aeff7cd95',
 4: 'd5f50673-d7d4-43f1-9887-743092c292ed',
 5: '62e62e94-6e3f-4c7c-b642-6cedf3c6de25',
 6: '9af20004-83e7-4a49-b9fb-5dc3e800ec81',
 7: '139be4c7-a2e1-4e69-83d3-5ce519513e5d',
 8: '52a32ec1-4d13-41ea-baf1-3f08043590ea',
 9: 'ba5a3985-b875-4b2d-be14-97a75bc2bf63',
 10: 'b37bd3e8-7c15-4abd-ac94-2aa7daf616ce',
 11: '51777860-fed1-40a5-ac9b-320a76c52776',
 12: 'c07da825-085b-4f70-924d-9ae52aa22367',
 13: '3998b044-54ca-4a7d-b312-3902b561c100',
 14: '3205a01a-03fb-4b14-ad91-18c62b85c8f5',
 15: '9c951c4e-b4f4-4bee-93cb-519de45b48e4',
 16: 'fa3ac1bb-526b-404e-a4ea-599321f22fa5'}

In [10]:
vector_store.get_by_ids(['189e466c-28da-45c5-bb7c-bb7218d7edba'])

[]

## Step 2 - Retrieval

In [11]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [12]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027606152680>, search_kwargs={'k': 4})

In [29]:
retriever.invoke('Describe about the method of cheating for coding interviews.')

[Document(id='638a0580-2292-4e78-aeb3-9b21f9051a47', metadata={}, page_content="prepare for, what to expect in the interview, and as long as they worked on this skill of solving leak code problems, they were set. So, it seemed perfect until it wasn't. So, the first problem with leaked code was that cheating was huge. Specifically after 2020, when everything went virtual, people went crazy with cheating. People used things like chatbt, got on secret Discord calls. There were even computer vision tools that could change the orientation of the appearance of your eyes. So even if you're looking to the side, like a separate monitor that might have like a solution there. It'll appear as if you're always looking straight at the interviewer. But there were always flaws in how it would work. Whether it be cheating software glitching or chatbt giving an incorrect answer, some small element that would make the tool unusable. Until things took a huge turn. In winter 2025, a Colombia student named 

## Step 3 - Augmentation

In [14]:
prompt = PromptTemplate(
    template="""
        You are a helpful assistant.
        Answer ONLY from the provided transcript context.
        If the context is insufficient, just say you don't know.    # answer-grounding to prevent halucination

        {context}
        Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [15]:
question          = "is the topic of dsa vs system design and project building discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [16]:
retrieved_docs

[Document(id='3998b044-54ca-4a7d-b312-3902b561c100', metadata={}, page_content="design is pretty much a type of interview that is traditionally given to senior level engineers. And the questions could go something like, if you had to design the architecture of a company like Netflix or Spotify, how would you go about it? Someone might start by asking, would you use AWS as your cloud provider? And then dive into choices around storage, latency, availability zones, and caching strategies. Then they might follow up with a scenario like, what would you do if there's a sudden traffic spike? to which someone would respond by saying you would scale horizontally using load balancers, autoscaling groups, and possibly a content delivery network to distribute the load efficiently. And although sure, you could technically cheat in this, it's much more difficult because it's more of a conversation where you're talking about the design of things, not just here's a question and here's an answer. Plus

In [17]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"design is pretty much a type of interview that is traditionally given to senior level engineers. And the questions could go something like, if you had to design the architecture of a company like Netflix or Spotify, how would you go about it? Someone might start by asking, would you use AWS as your cloud provider? And then dive into choices around storage, latency, availability zones, and caching strategies. Then they might follow up with a scenario like, what would you do if there's a sudden traffic spike? to which someone would respond by saying you would scale horizontally using load balancers, autoscaling groups, and possibly a content delivery network to distribute the load efficiently. And although sure, you could technically cheat in this, it's much more difficult because it's more of a conversation where you're talking about the design of things, not just here's a question and here's an answer. Plus, there are a lot of follow-ups and intricacies. For each design choice in\n\nm

In [18]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [19]:
final_prompt

StringPromptValue(text="\n        You are a helpful assistant.\n        Answer ONLY from the provided transcript context.\n        If the context is insufficient, just say you don't know.    # answer-grounding to prevent halucination\n\n        design is pretty much a type of interview that is traditionally given to senior level engineers. And the questions could go something like, if you had to design the architecture of a company like Netflix or Spotify, how would you go about it? Someone might start by asking, would you use AWS as your cloud provider? And then dive into choices around storage, latency, availability zones, and caching strategies. Then they might follow up with a scenario like, what would you do if there's a sudden traffic spike? to which someone would respond by saying you would scale horizontally using load balancers, autoscaling groups, and possibly a content delivery network to distribute the load efficiently. And although sure, you could technically cheat in this

## Step 4 - Generation

In [20]:
client = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [21]:
answer = client.invoke(final_prompt)
print(answer.content)

Yes, the topic of system design and project building is discussed in this video. 

For system design, it is described as a type of interview traditionally given to senior level engineers, involving questions about designing the architecture of companies like Netflix or Spotify. The discussion includes considerations around cloud providers, storage, latency, availability zones, caching strategies, and how to handle traffic spikes. It emphasizes the conversational nature of these interviews, the need to defend design choices, and the possibility of curveball questions.

Regarding project building, it mentions simple project ideas like a Google calendar autosync tool, an event RSVP tracker, or a custom GPT for drafting emails and social posts. It highlights the importance of problem-solving skills over the actual code written, as that is what hiring managers are looking for. It also suggests that through solving real-world problems, one might accidentally create a startup.


## Building a Chain (FINALLY!)

In [22]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [23]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [24]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [25]:
# parallel_chain.invoke('why dsa is not enough?') # check - working

In [26]:
parser = StrOutputParser()

In [27]:
main_chain = parallel_chain | prompt | client | parser

In [28]:
main_chain.invoke('Describe about the method of cheating for coding interviews.')

"The method of cheating for coding interviews involved using a software called Interview Coder, created by a student named Roy Lee. This software was designed to allow users to cheat in real time during technical coding interviews. It worked by sitting on top of the user's screen, where the user could feed it a screenshot of a coding problem. The software would then process this input using a large language model (LLM) to generate a solution.\n\nTo avoid detection, the software also had a feature that moved the user's eyes around the screen to make it appear natural, preventing the interviewer from noticing that the user was not genuinely solving the problem. Roy demonstrated the effectiveness of this tool by using it in his own interviews at companies like Amazon, Meta, and TikTok, where he received high-paying job offers."

## Scope of Improvements

1. **UI based enhancements**
2. **Evaluation**
    - a. Ragas  
    - b. LangSmith  
3. **Indexing**
    - a. Document Ingestion  
    - b. Text Splitting  
    - c. Vector Store  
4. **Retrieval**
    - a. *Pre-Retrieval*  
        - i. Query rewriting using LLM  
        - ii. Multi-query generation  
        - iii. Domain aware routing  
    - b. *During Retrieval*  
        - i. MMR  
        - ii. Hybrid Retrieval  
        - iii. Reranking  
    - c. *Post-Retrieval*  
        - i. Contextual Compression  
5. **Augmentation**
    - a. Prompt Templating  
    - b. Answer grounding  
    - c. Context window optimization  
6. **Generation**
    - a. Answer with Citation  
    - b. Guard railing  
7. **System Design**
    - a. Multimodal  
    - b. Agentic  
    - c. Memory based  
